# Introduction to Quantum Error Correction using Repetition Codes

## Introduction

量子コンピューティングでは、情報をqubitでエンコードする必要があります。過去数十年にわたって開発されたほとんどの量子アルゴリズムは、これらのqubitが完全であると想定しています。それらは、私たちが望む任意の状態で準備でき、完全な制度で操作できます。これらの仮定に従うqubitは、しばしば"logical qubits"(論理キュービット)と呼ばれます。

過去数十年は、より良い品質のqubitが常に開発されており、qubitとして振る舞う物理システムの発見においても大きな進歩を遂げてきました。ただし、欠陥を完全に取り除くことはできません。これらのqubtiは常に不正確すぎて、論理qubitとして直接使用することはできません。代わりに、それらを"physical qubit"(物理キュービット)と呼びます。

量子コンピューティングの現在の時代では、カスタムアルゴリズムを設計し、エラー軽減効果を使用することにより、欠陥があっても物理qubitを使用しようとしています。ただし、フォールトトレランスの将来の時代には、物理qubitから論理qubitを構築する方法を見つける必要があります。これは、量子エラー訂正のプロセスを通じて行われます。このプロセスでは、論理qubitが多数の物理qubitにエンコードされます。エンコーディングは、物理的なqubitを非常に絡み合う回路に常に通すことで維持されます。エラーの兆候を検出し、その影響を取り除くために、"Auxilliary degrees of freedom"(補助自由度)も常に測定されます。量子計算の実装に必要な論理qubitの操作は、基本的にこの手順に小さな摂動を加えることによって実行されます。

このプロセスには膨大な労力が必要となるため、フォールトトレラントな量子コンピュータで実行されるほとんどの操作は、エラーの検出と修正の目的で実行されます。従って、フォールトトレラントな量子計算に向けた進捗状況をベンチマークするときは、デバイスがエラー修正をどれだけ適切に実行しいているかを追跡する必要があります。

この章では、エラー訂正の特定の例である"the repetition code"(反復コード)について見ていきます。量子エラー訂正の真の例ではありませんが、qubitではなく物理qubitを使用して論理ビットをエンコードします。これは、量子エラー訂正コードの全ての基本概念の簡単なガイドとして機能します。また、現在のプロトタイプデバイスでどのように実行できるかについても説明します。

## Introduction to the repetition code

### The basics of error correction

エラー訂正の背後にある基本的な考え方は、量子情報についても古典情報についても同じです。これにより、電話で話すという非常に簡単な例を考えることから始めることができます。答えが「はい」または「いいえ」である質問を誰かがあなたに尋ねた場合、あなたの応答を与える方法は、2つの要因によって異なります。

　・あなたが正しく理解されていることはどのくらい重要か？<br>
　・あなたの通信状況はどのくらい良いか？

これらの両方は、確率でパラメーター化できます。1つ目は、誤解される最大許容確率$P_a$を使用できます。アイスクリームの味の好みを確認するように求められている場合、チョコレートではなくバニラを食べても構わないのであれば、$P_a$はかなり高いのかもしれません。ただし、誰かの命がかかっている質問をされている場合、$P_a$は遥かに低くなります。

2つ目は、接続が悪いために回答が文字化けする確率$p$を使用できます。簡単にするために、文字化けした「はい」が単にナンセンスのように聞こえるのではなく、「いいえ」のように聞こえる場合を想像してみましょう。同様に、「いいえ」は
「はい」に変換されます。つまり、$p$は完全に誤解されている確率です。

良い通信状況である場合、または比較的重要でない質問は$p＜P_a$になります。この場合、可能な限り最も直接的な方法で単純に答えても問題はありません。「はい」または「いいえ」と言うだけです。

ただし、通信状態が悪く、回答が重要な場合は、$p>P_a$と表示されます。この場合、単一の「はい」または「いいえ」では不十分です。誤解される可能性が高すぎます。代わりに、より複雑な構造で回答をエンコードする必要がああります。これにより、メッセージが破壊される可能性があっても、受信者は意味を解読できます。最も単純な方法は、多くの人が考えずに行う方法です。答えを何度も繰り返すだけです。例えば、「はい」の代わりに「はい、はい、はい」または「いいえ」の代わりに「いいえ、いいえ」と言います。

この場合、受信者が「はい、はい、はい」と聞くと、もちろん、送信者が「はい」を意味したと結論づけます。彼らが「いいえ、はい、はい」、「はい、いいえ、はい」、「はい、はい、いいえ」を聞いた場合、答えには否定よりも積極的であるため、おそらく同じことを結論づけます。この場合に誤解するには、少なくとも2つの返信が文字化けする必要があります。この確率$P$は$p$未満になります。従って、この方法でエンコードすると、メッセージが理解される可能性が高くなります。以下のコードセルは、この例を示しています。

In [1]:
p1 = 0.01
p3 = 3 * p1**2 * (1-p1) + p1**3 # probability of 2 or 3 errors
print('Probability of a single reply being garbled: {}'.format(p1))
print('Probability of a the majority of three replies being garbled: {:.4f}'.format(p3))

Probability of a single reply being garbled: 0.01
Probability of a the majority of three replies being garbled: 0.0003


$P<P_a$の場合、この手法は問題を解決します。そうでない場合は、単純に繰り返しを追加できます。上記の$P<p$であるという事実は、過半数を反転するために少なくとも2つの応答が文字化けする必要があるという事実から来ているため、最も可能性の高い可能性でさえ、$〜p_2$の確率があります。5回の繰り返しの場合、過半数を反転させるには、少なくとも3つの応答が文字化けする必要があります。これは、確率$〜p_3$で発生します。この場合、$p$の値はさらに低くなります。実際、繰り返しの数を増やすと、$P$は指数関数的に減少します。接続がどれほど悪い場合でも、メッセージが正しく伝わるようにする必要がある場合でも、答えを十分に繰り返すだけでそれを実現できます。

これは簡単な例ですが、エラー訂正の全ての側面が含まれています。

　・送信または保存される情報がいくつかあります。<br>
　　(この場合、「はい」または「いいえ」です。)<br>
　・情報はノイズから保護するために、より多いなシステムでエンコードされます。<br>
　　 (この場合、メッセージを繰り返すことにより)<br>
　・情報は最終的にデコードされ、ノイズの影響を軽減します。<br>
　　(この場合、送信されたメッセージの大部分を信頼することによって。)

これと同じエンコードスキームは、「はい」と「いいえ」を0と1に置き換えるだけで、バイナリにも使用できます。従って、状態$|0 \rangle $および$|1 \rangle$を使用することにより、qubitを簡単に一般化することもできます。どちらの場合も、反復コードと呼ばれます。他の多くの形式のエンコーディングも、古典的な場合と量子の場合の両方で可能であり、反復コードよりも多くの点で優れています。ただし、最も単純なエンコーディングとしてのステータスは、特定のアプリケーションに役立ちます。1つは、量子エラー訂正の背後にあるアイデアを実装する最初で最も単純なテストとして、まさにQiskitで使用されるものです。。

### Correcting errors in qubits

Qiskitを使用して、これらのアイデアを明示的に実装します。不完全なqubitの影響を確認するには、プロトタイプデバイスのqubitを使用するだけです。シミュレーションで効果を再現することもできます。以下の関数は、これを行うために単純なノイズモデルを作成します。これらは、確率$p$で発生する単一のノイズイベントの前述の単純なケースを超えています。代わりに、発生する可能性のある2つの形式のエラーを検討します。1つはゲートエラーです。これは、実行する操作の不完全さです。ここでは、いわゆる"depolarizing noise"(減極ノイズ)を使用して、これを簡単な方法でモデル化します。これの効果は、$P_{gate}$で、任意のqubitの状態を完全にランダムな状態に置き換えることです。2つの量子ビットゲートの場合、各qubitに個別に適用されます。もう1つのノイズは、測定の際に起こるノイズです。これは、確率$p_{meas}$で測定する直前に、0から1に、またはその逆に反転します。

In [2]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_meas,p_gate):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

これにより、エラーのタイプごとに1%の確率でこのようなノイズモデルを作成します。

In [3]:
noise_model = get_noise(0.01,0.01)

状態$|0\rangle$の3つのqubitを使用して0を格納しようとしたときに、これがどのような影響を与えるかを見てみましょう。プロセスショット=1024回を繰り返して、異なる結果の可能性を確認します。

In [5]:
from qiskit import QuantumCircuit, execute, Aer

qc0 = QuantumCircuit(3,3,name='0') # initialize circuit with three qubits in the 0 state

qc0.measure(qc0.qregs[0],qc0.cregs[0]) # measure the qubits

# run the circuit with th noise model and extract the counts
counts = execute( qc0, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

{'100': 14, '010': 12, '000': 990, '001': 8}


ここでは、ノイズがない場合と同様に、ほとんど全ての結果が"000"のままであることがわかります。残りの可能性のうち、0を過半数に持つ可能性が最も高いです。合計すると、100未満のサンプルが1の過半数で取得されます。この回路を使用して0をエンコードする場合、これは$P<1$%となります。

次に、状態$|1>$の3つのqubitを使用して1を格納する場合も同じように試してみましょう。

In [6]:
qc1 = QuantumCircuit(3, 3, name='0') # initialize circuit with three qubits in the 0 state
qc1.x(qc1.qregs[0]) # flip each 0 to 1

qc1.measure(qc1.qregs[0],qc1.cregs[0]) # measure the qubits

# run the circuit with th noise model and extract the counts
counts = execute(qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

{'011': 14, '111': 983, '101': 15, '110': 12}


過半数が間違った状態(この場合は0)で出力されるサンプルの数は、100を遥かに下回るので、$P<1$%です。0と1のどちらを格納するかにかかわらず、どちらのノイズソースよりもエラーの可能性が少ない情報を取得できます。

これは、検討したノイズが比較的弱いために可能でした。$p_{meas}$と$p_{gate}$を増やすと、確率$P$が高くなります。これの極端なケースは、どちらかがビットフリップエラー$x$を適用する可能性が50/50の場合です。例えば、前と同じ回路を実行してみましょう。ただし、$p_{meas}=0.5$および$p_{gate}=0$とします。

In [7]:
noise_model = get_noise(0.5,0.0)
counts = execute(qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()
print(counts)

{'011': 136, '010': 138, '110': 116, '001': 136, '100': 123, '101': 122, '000': 118, '111': 135}


このノイズでは、全ての結果が等しい確率で発生し、結果の違いは統計的ノイズのみによるものです。エンコードされた状態の痕跡は残りません。これは、エラー訂正のために考慮すべき重要なポイントです。時々、ノイズが強すぎて訂正できない場合があります。最適な方法は、必要な情報をエンコードする適切な方法と、ノイズがそれほど強くないハードウェアを組み合わせることです。

### Storing qubits

これまで、エンコーディングとデコーディングの間に遅延がない場合について検討してきました。qubitの場合、これは、回路の初期化と最終測定の実行の間に経過する有意な時間がないことを意味します。

ただし、大幅な遅延が発生する場合が多くあります。明白な例として、量子ハードドライブのように、量子状態をエンコードして長期間保存したい場合があります。それほど明白ではありませんが、遥かに重要な例は、フォールトトレラントな量子計算自体の実行です。このため、量子状態を保存し、計算中にそれらの整合性を維持する必要があります。これはまた、格納された情報を必要な方法で操作できるようにする必要があり、操作の実行時に発生する可能性のあるエラーを修正します。

全ての場合において、エラーは、何かが起こったとき(ゲートや測定など)だけでなく、qubitがアイドル状態の時にも発生するという事実を考慮する必要があります。このようなノイズは、qubitが相互に、またそれらの環境と相互作用するという事実によるものです。qubitをアイドル状態にしておく時間が長いほど、このノイズの影響は大きくなります。十分に長く放置すると、上記の$P_{meas}=0.5$の場合のように、ノイズが強すぎてエラーを確実に修正できない状況が発生します。

解決策は、ずっと測定し続けることです。qubitが長時間アイドル状態のままになることはありません。代わりに、発生したエラーを追跡するために、システムから常に情報が抽出されます。

単に0または1を格納したいという古典的な情報の場合、これは各qubitの値を常に測定するだけで実行できます。ノイズによって値がいつ変化するかを追跡することで、エラーが発生した時の履歴を簡単に推測できます。

しかし、量子情報についてはそれほど簡単ではありません。例えば、論理状態$|+\rangle$をエンコードする場合を考えます。私たちのエンコーディングは

$|0\rangle \rightarrow |000\rangle,\,\,\,\,|1\rangle \rightarrow |111\rangle$

従って、論理$|+\rangle$状態をエンコードするには、

$|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)\rightarrow\frac{1}{\sqrt{2}}(|000\rangle + |111\rangle).$

私たちが使用している繰り返しエンコーディングでは、論理qubitのz測定("0"状態と"1"状態を区別)は、各物理qubitのz測定を使用して行われます。論理測定の最終結果は、どの出力が過半数に含まれるかを調べるだけで、物理qubit測定結果からデコードされます。

先に述べたように、物理qubitのz測定を常に実行することにより、長期間保存されている論理qubitのエラーを追跡できます。ただし、これは物理qubitのz測定を常に実行することに実質的に対応していることに注意してください。単に0または1を格納している場合はこれで問題ありませんが、重ね合わせを格納している場合は望ましくない影響があります。具体的には、このようなエラーチェックを初めて行う時に、重ね合わせを折りたたみます。

これは理想的ではありません。論理qubitで計算を実行したい場合、または最終測定の前に基底変換を実行したい場合は、重ね合わせを保持する必要があります。破棄するとエラーになります。ただし、これはデバイスの欠陥が原因のエラーではありません。これは、エラーを修正する試みの一環として導入したエラーです。また、量子コンピュータに保存されている任意の重ね合わせを再現することは望めないため、修正できないエラーです。

このため、論理qubitが長期保存されている時に発生するエラーを追跡する別の方法を見つける必要があります。これにより、エラーを検出して修正し、最終的な測定結果を高い確率でデコードするために必要な情報が得られます。ただし、保存する必要がある重ね合わせを折りたたむことにより、プロセス中に修正不可能なエラーが発生することはありません。

これを行う方法は、次の回路要素を使用することです。

In [8]:
from qiskit import QuantumRegister, ClassicalRegister

cq = QuantumRegister(2,'code_qubit')
lq = QuantumRegister(1,'ancilla_qubit')
sb = ClassicalRegister(1,'syndrome_bit')
qc = QuantumCircuit(cq,lq,sb)
qc.cx(cq[0],lq[0])
qc.cx(cq[1],lq[0])
qc.measure(lq,sb)
qc.draw()

code_qubit_0: ──■──────────
                   │          
   code_qubit_1: ──┼────■─────
                 ┌─┴─┐┌─┴─┐┌─┐
ancilla_qubit_0: ┤ X ├┤ X ├┤M├
                 └───┘└───┘└╥┘
 syndrome_bit_0: ═══════════╩═

ここには3つの物理qubitがあります。2つは"code qubits"(コードキュービット)と呼ばれ、もう1つは"ancilla qubit"(アンシラキュービット)と呼ばれます。"syndrome bit"(シンドロームビット)と呼ばれる出力の1ビットが抽出されます。補助量子ビットは常に状態"0"で初期化されます。ただし、コードキュービットはさまざまな状態で初期化できます。さまざまな入力が出力にどのような影響を与えるかを確認するには、ある状態でコードキュービットを準備する回路qc_initを作成し、回路qc_init + qcを実行します。

まず、自明なケース：qc_initは何もしないため、コードキュービットは最初は$|00\rangle$です。

In [9]:
qc_init = QuantumCircuit(cq)

(qc_init+qc).draw()

code_qubit_0: ──■──────────
                   │          
   code_qubit_1: ──┼────■─────
                 ┌─┴─┐┌─┴─┐┌─┐
ancilla_qubit_0: ┤ X ├┤ X ├┤M├
                 └───┘└───┘└╥┘
 syndrome_bit_0: ═══════════╩═

In [10]:
counts = execute( qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

Results: {'0': 1024}


全ての場合において、結果は0です。

次に、初期状態の$|11\rangle$を試してみましょう。

In [11]:
qc_init = QuantumCircuit(cq)
qc_init.x(cq)

(qc_init+qc).draw()

┌───┐             
   code_qubit_0: ┤ X ├──■──────────
                 ├───┤  │          
   code_qubit_1: ┤ X ├──┼────■─────
                 └───┘┌─┴─┐┌─┴─┐┌─┐
ancilla_qubit_0: ─────┤ X ├┤ X ├┤M├
                      └───┘└───┘└╥┘
 syndrome_bit_0: ════════════════╩═

In [12]:
counts = execute(qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

Results: {'0': 1024}


この場合の結果も常に0です。量子力学の線形性を考えると、以下の例のように、$|00\rangle$と$|11\rangle$の重ね合わせについても同じことが当てはまると予想できます。

In [13]:
qc_init = QuantumCircuit(cq)
qc_init.h(cq[0])
qc_init.cx(cq[0],cq[1])

(qc_init+qc).draw()

┌───┐                  
   code_qubit_0: ┤ H ├──■────■──────────
                 └───┘┌─┴─┐  │          
   code_qubit_1: ─────┤ X ├──┼────■─────
                      └───┘┌─┴─┐┌─┴─┐┌─┐
ancilla_qubit_0: ──────────┤ X ├┤ X ├┤M├
                           └───┘└───┘└╥┘
 syndrome_bit_0: ═════════════════════╩═

In [14]:
counts = execute(qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

Results: {'0': 1024}


逆の結果は、"01"、"10"またはそれらの任意の重ね合わせの初期状態で見つかります。

In [15]:
qc_init = QuantumCircuit(cq)
qc_init.h(cq[0])
qc_init.cx(cq[0],cq[1])
qc_init.x(cq[0])

(qc_init+qc).draw()

┌───┐     ┌───┐             
   code_qubit_0: ┤ H ├──■──┤ X ├──■──────────
                 └───┘┌─┴─┐└───┘  │          
   code_qubit_1: ─────┤ X ├───────┼────■─────
                      └───┘     ┌─┴─┐┌─┴─┐┌─┐
ancilla_qubit_0: ───────────────┤ X ├┤ X ├┤M├
                                └───┘└───┘└╥┘
 syndrome_bit_0: ══════════════════════════╩═

In [16]:
counts = execute(qc_init+qc, Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

Results: {'1': 1024}


このような場合、出力は常に"1"です。

従って、この測定は、複数のqubitの集合的な特性について教えてくれます。具体的には、2つのコードキュービットを調べ、それらの状態がz基底で同じか異なるかを判断します。$|00\rangle$や$|11\rangle$のようにz基底が同じ基底状態の場合、測定値は単に0を返します。これらの重ね合わせについても同様です。これらの状態をどのようにも区別しないため、このような重ね合わせが崩れることもありません。

同様に、z基底が異なる基底状態の場合、1を返します。これは、"$|01\rangle$"、"$|10\rangle$"またはそれらの任意の重ね合わせで発生します。

ここで反復コードの物理qubitの全てのペアにこのような"syndrome measurement"(シンドローム測定)を適用するとします。それらの状態が繰り返された$|0\rangle$、繰り返された$|1\rangle$またはそれらの任意の重ね合わせによって記述されている場合、全てのシンドローム測定値は0を返します。この結果から、実際の状態は繰り返し状態でエンコードされていることがわかります。私たちがそれらになりたいこと、そしてエラーが発生していないと推定することができます。ただし、一部のシンドローム推定値が1を返す場合、それはエラーのサインです。従って、これらの測定結果を使用して、結果をデコードする方法を決定できます。

### Quantm repetition code

これで、反復コードの量子バージョンがどのように実装されているかを正確に理解するのに十分です。

Ignisから必要なツールをインポートして、Qiskitで使用できます。

In [17]:
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import lookuptable_decoding
from qiskit.ignis.verification.topological_codes import GraphDecoder

論理qubitをエンコードする物理qubitの数を自由に選択できます。また、最終的な読み出し測定の前に、論理qubitを保存する間にシンドローム測定を適用する回数を選択することもできます。最も重要なケースである、3つの繰り返しと1つのシンドローム測定ラウンドから始めましょう。その後、Qiskit-IgnisのRepetitionCodeオブジェクトを使用して、繰り返しコードの回路を自動的に作成できます。

In [18]:
n = 3
T = 1

code = RepetitionCode(n,T)

これにより、コードに使用されるキュービットレジスタの名前や補助qubitなど、コードのさまざまなプロパティを検査できます。

RepetitionCodeには、コードを実装する2つの量子回路が含まれています。可能な2つの論理ビット値のそれぞれに1つです。以下は、それぞれ論理0と論理1の場合です。

In [19]:
code.circuit['0'].draw()

┌───┐     ┌───┐        ┌─┐         
      link_qubit_0: ┤ X ├─────┤ X ├────────┤M├─────────
                    └─┬─┘┌───┐└─┬─┘┌───┐   └╥┘┌─┐      
      link_qubit_1: ──┼──┤ X ├──┼──┤ X ├────╫─┤M├──────
                      │  └─┬─┘  │  └─┬─┘┌─┐ ║ └╥┘      
      code_qubit_0: ──■────┼────┼────┼──┤M├─╫──╫───────
                           │    │    │  └╥┘ ║  ║ ┌─┐   
      code_qubit_1: ───────■────■────┼───╫──╫──╫─┤M├───
                                     │   ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ─────────────────■───╫──╫──╫──╫─┤M├
                                         ║  ║  ║  ║ └╥┘
round_0_link_bit_0: ═════════════════════╬══╩══╬══╬══╬═
                                         ║     ║  ║  ║ 
round_0_link_bit_1: ═════════════════════╬═════╩══╬══╬═
                                         ║        ║  ║ 
        code_bit_0: ═════════════════════╩════════╬══╬═
                                                  ║  ║ 
        code_bit_1: ══════════════════════════════╩══╬═
                                                     ║ 
        code_bit_2: ═════════════════════════════════╩═

In [20]:
code.circuit['1'].draw()

┌───┐     ┌───┐        ┌─┐         
      link_qubit_0: ─────┤ X ├─────┤ X ├────────┤M├─────────
                         └─┬─┘┌───┐└─┬─┘┌───┐   └╥┘┌─┐      
      link_qubit_1: ───────┼──┤ X ├──┼──┤ X ├────╫─┤M├──────
                    ┌───┐  │  └─┬─┘  │  └─┬─┘┌─┐ ║ └╥┘      
      code_qubit_0: ┤ X ├──■────┼────┼────┼──┤M├─╫──╫───────
                    ├───┤       │    │    │  └╥┘ ║  ║ ┌─┐   
      code_qubit_1: ┤ X ├───────■────■────┼───╫──╫──╫─┤M├───
                    ├───┤                 │   ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ┤ X ├─────────────────■───╫──╫──╫──╫─┤M├
                    └───┘                     ║  ║  ║  ║ └╥┘
round_0_link_bit_0: ══════════════════════════╬══╩══╬══╬══╬═
                                              ║     ║  ║  ║ 
round_0_link_bit_1: ══════════════════════════╬═════╩══╬══╬═
                                              ║        ║  ║ 
        code_bit_0: ══════════════════════════╩════════╬══╬═
                                                       ║  ║ 
        code_bit_1: ═══════════════════════════════════╩══╬═
                                                          ║ 
        code_bit_2: ══════════════════════════════════════╩═

これらの回路では、2種類の物理qubitがあります。物理状態でエンコードされる3つの物理qubitである"code qubits"(コードキュービット)があります。シンドローム測定の補助qubitとして機能する"link qubits"(リンク量子ビット)もあります。

これらの回路での1回のシンドローム測定は、2つのシンドローム測定のみで構成されています。1つはコードキュービット0と1を比較し、もう1つはコードキュービット1と2を比較してさらに測定する必要があることが予想されます。ただし、これらの2つで十分です。これは、0と2のz基底状態が同じかどうかに関する情報は、0と1に関する同じ情報から、1と2の情報から推測できるためです。実際$n-qubit$の場合、必要な情報はnだけから取得できます。qubitの隣接ペアの

In [21]:
def get_raw_results(code,noise_model=None):

    circuits = code.get_circuit_list()
    raw_results = {}
    for log in range(2):
        job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model)
        raw_results[str(log)] = job.result().get_counts(str(log))
    return raw_results

raw_results = get_raw_results(code)
for log in raw_results:
    print('Logical',log,':',raw_results[log],'\n')

Logical 0 : {'000 00': 1024} 

Logical 1 : {'111 00': 1024} 



In [22]:
code = RepetitionCode(n,4)

raw_results = get_raw_results(code)
for log in raw_results:
    print('Logical',log,':',raw_results[log],'\n')

Logical 0 : {'000 00 00 00 00': 1024} 

Logical 1 : {'111 00 00 00 00': 1024} 



In [23]:
code = RepetitionCode(5,4)

raw_results = get_raw_results(code)
for log in raw_results:
    print('Logical',log,':',raw_results[log],'\n')

Logical 0 : {'00000 0000 0000 0000 0000': 1024} 

Logical 1 : {'11111 0000 0000 0000 0000': 1024} 



### Lookup table decoding

In [24]:
code = RepetitionCode(3,1)

noise_model = get_noise(0.05,0.05)

raw_results = get_raw_results(code,noise_model)
for log in raw_results:
    print('Logical',log,':',raw_results[log],'\n')

Logical 0 : {'010 11': 4, '010 10': 4, '010 00': 39, '110 00': 4, '110 01': 2, '001 10': 4, '000 01': 81, '100 00': 49, '000 00': 674, '011 00': 4, '000 11': 3, '101 00': 4, '100 10': 5, '000 10': 63, '010 01': 16, '001 00': 57, '100 01': 5, '001 01': 4, '011 01': 2} 

Logical 1 : {'010 11': 1, '111 10': 60, '101 10': 4, '111 01': 58, '110 00': 47, '010 00': 6, '010 10': 2, '110 01': 16, '111 11': 11, '001 10': 1, '000 01': 1, '100 00': 6, '011 00': 47, '111 00': 621, '101 01': 26, '101 00': 44, '001 11': 2, '100 10': 1, '101 11': 18, '011 10': 16, '110 10': 7, '110 11': 2, '100 11': 1, '100 01': 3, '001 00': 6, '001 01': 5, '011 11': 4, '011 01': 8} 



In [25]:
circuits = code.get_circuit_list()
table_results = {}
for log in range(2):
    job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=10000 )
    table_results[str(log)] = job.result().get_counts(str(log))

In [26]:
P = lookuptable_decoding(raw_results,table_results)
print('P =',P)

P = {'0': 0.0237, '1': 0.0237}


### Graph theoretic decoding

In [27]:
code = RepetitionCode(3,2)

raw_results = get_raw_results(code,noise_model)

results = code.process_results( raw_results )

for log in ['0','1']:
    print('\nLogical ' + log + ':')
    print('raw results       ', {string:raw_results[log][string] for string in raw_results[log] if raw_results[log][string]>=50 })
    print('processed results ', {string:results[log][string] for string in results[log] if results[log][string]>=50 })


Logical 0:
raw results        {'000 10 00': 56, '000 00 01': 54, '000 00 00': 481, '000 01 00': 52}
processed results  {'0 0  00 10 10': 56, '0 0  01 01 00': 54, '0 0  00 00 00': 481, '0 0  00 01 01': 52}

Logical 1:
raw results        {'111 00 00': 462, '111 10 00': 51}
processed results  {'1 1  00 00 00': 462, '1 1  00 10 10': 51}


## Running a repetition code benchmarking procedure

次に、実際のデバイスで繰り返しコードの例を実行し、その結果をベンチマークとして使用します。最初に、プロセスを簡単に要約します。これは、この反復コードの例に適用されますが、topological_codesの他のベンチマーク手順にも当てはまり、Qiskit Ignis全般にも当てはまります。いずれの場合も、次の3つのステップのプロセスが使用されます。

1. タスクが定義されました。Qiskit Ignisは、実行する必要のある一連の回路を決定して作成します。
2. 回路が実行されます。これは通常、Qiskitを使用して行われます。ただし、原則として、任意のサービスまたは実験装置を接続できます。
3. Qiskit Ignisは、回路からの結果を処理して、特定のタスクに必要な出力を作成するために使用されます。

topological_codesの場合、ステップ1では、量子謝り訂正コードのタイプとサイズを選択する必要があります。各タイプのコードには専用のPythonクラスがあります。対応するオブジェクトは、RepetitionCodeオブジェクトのnやTなど、必要なパラメーターを提供することによって初期化されます。結果のオブジェクトには、単純な論理キュービット状態($|0\rangle$や$|1\rangle$など)をエンコードする特定のコードに対応する回路が含まれており、最終的な読み出しの前に、指定されたラウンド数のエラー検出手順を実行します。簡単な論理的根拠

In [28]:
step_2 = False
step_3 = False

実際のデバイスをベンチマークするには、クラウド経由でそのデバイスにアクセスし、そのデバイスでの実行に適した回路をコンパイルするために必要なツールが必要です。これらは次のようにインポートされます。

In [29]:
from qiskit import IBMQ
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

これで、回路の実行に使用されるバックエンドオブジェクトを作成できます。これは、デバイスを指定するために使用される文字列を提供することによって行われます。ここでは、"ibmq_16_melbourne"が使用されています。これには、執筆時点でアクティブなqubitが15個あります。また、"ibmq_rochester"で指定されている53qubitロチェスターデバイスについても検討します。

In [30]:
device_name = 'ibmq_16_melbourne'

if step_2:
    
    IBMQ.load_account()
    
    for provider in IBMQ.providers():
        for potential_backend in provider.backends():
            if potential_backend.name()==device_name:
                backend = potential_backend

    coupling_map = backend.configuration().coupling_map

実際のデバイスで回路を実行する場合、トランスパイレーションプロセスが最初に実装されます。これにより、回路のゲートがデバイスによってネイティブゲートセットの実装に変更されます。場合によっては、これらの変更はかなり簡単です。例えば、各アダマールを、対応するオイラー角による単一のqubit回転として表現します。ただし、回路がデバイスの接続を考慮しない場合、変更はより大きなものに名ある可能性があります。例えば、回路がデバイスによって直接実装されていないcontroll-NOTを必要とするとします。次に、追加のcontroll-NOTゲートを使用してqubit状態を移動するなどの手法を使用して、効果を再現する必要があります。これにより、追加のノイズが導入されるだけでなく、既に存在するノイズも非局所化されます。元の回路の単一のqubitエラーは、追加の熱分解の作用下でマルチqubitの怪物になる可能性があります。従って、量子エラー訂正回路を実行するときは、そのような自明でない変換を防止する必要があります。

繰り返しコードのテストでは、qubitを線に沿って効果的に配列する必要があります。必要な唯一の制御されたNOTゲートは、そのラインに沿った隣同士の間です。従って、私たちの最初の仕事は、デバイスの結合マップを調査し、ラインを見つけることです。

メルボルンでは、15qubit全てをカバーするラインを見つけることができます。以下のリストの行で指定されている選択肢は、エラーが発生しやすいcxゲートを回避するように設計されています。53qubitロチェスターデバイスの場合、53qubit全てをカバーする単一のラインはありません。代わりに43をカバーする次の選択肢を使用できます。

In [31]:
if device_name=='ibmq_16_melbourne':
    line = [13,14,0,1,2,12,11,3,4,10,9,5,6,8,7]
elif device_name=='ibmq_rochester':
    line = [10,11,17,23,22,21,20,19,16,7,8,9,5]#,0,1,2,3,4,6,13,14,15,18,27,26,25,29,36,37,38,41,50,49,48,47,46,45,44,43,42,39,30,31]

これで、アクセスできるqubitの数がわかったので、実行する各コードの繰り返しコードオブジェクトを作成できます。繰り返しが$n$回のコードでは、$n$個のコード量子ビットと$n-1$個のリンク量子ビットが使用されるため、合計で$2n-1$個になることに注意してください。

In [32]:
n_min = 3
n_max = int((len(line)+1)/2)

code = {}

for n in range(n_min,n_max+1):
    code[n] = RepetitionCode(n,1)

これらのコードから回路を実行する前に、トランスパイラーが使用するデバイス上の物理qubitを認識していることを確認する必要があります。小kれは、line[0]のqubitを最初のコード量子ビットとして機能させるために使用し、line[1]のqubitを最初のリンク量子ビットとして使用することを意味します。これは、反復コードオブジェクトとラインを受け取り、コードのどのqubitがラインのどの要素に対応するかを指定するPython辞書を作成する次の関数によって実行されます。

In [33]:
def get_initial_layout(code,line):
    initial_layout = {}
    for j in range(n):
        initial_layout[code.code_qubit[j]] = line[2*j]
    for j in range(n-1):
        initial_layout[code.link_qubit[j]] = line[2*j+1]
    return initial_layout

これで回路をトランスパイルして、デバイスによって実際に実行される回路を作成できます。qubitの数を増やすことによって、蒸散が実際に重要な酷化をもたらしていないことを確認するためのチェックも行われます。さらに、コンパイルされた回路は1つのリストにまとめられ、同じバッチジョブで一度に送信できます。

In [34]:
if step_2:
    
    circuits = []
    for n in range(n_min,n_max+1):
        initial_layout = get_initial_layout(code[n],line)
        for log in ['0','1']:
            circuits.append( transpile(code[n].circuit[log], backend=backend, initial_layout=initial_layout) )
            num_cx = dict(circuits[-1].count_ops())['cx']
            assert num_cx==2*(n-1), str(num_cx) + ' instead of ' + str(2*(n-1)) + ' cx gates for n = ' + str(n)

これでジョブを実行する準備ができました。既に検討されているシミュレートされたジョブと同様に、この結果はdictionary raw_resultsに抽出されます。ただし、この場合異なるコードサイズの結果を保持するように拡張されます。つまり、以下のraw_results[n]は、特定のnに対して、以前に使用されたraw_results dictionaryの1つと同等です。

In [35]:
if step_2:
    
    job = execute(circuits,backend,shots=8192)

    raw_results = {}
    j = 0
    for d in range(n_min,n_max+1):
        raw_results[d] = {}
        for log in ['0','1']:
            raw_results[d][log] = job.result().get_counts(j)
            j += 1

データをファイルに保存すると、ステップ3の処理を後で実行したり繰り返したりできるので便利です。

In [36]:
if step_2: # save results
    with open('results/raw_results_'+device_name+'.txt', 'w') as file:
        file.write(str(raw_results))
elif step_3: # read results
    with open('results/raw_results_'+device_name+'.txt', 'r') as file:
        raw_results = eval(file.read())

以前に見たように、シンドロームを正しい形式で表現するためには、最初にデコードのプロセスで結果を書き換える必要があります。そのため、各反復コードオブジェクトcode[n]のprocess_resultsメソッドを使用して、各rax_results[n]から結果dictionary_results[n]を作成します。

In [37]:
if step_3:
    results = {}
    for n in range(n_min,n_max+1):
        results[n] = code[n].process_results( raw_results[n] )

デコードでは、コードごとに、GraphDecoderオブジェクトを設定する必要もあります。これらの初期化には、前のセクションで説明したように、シンドロームに対応するグラフの作成が含まれます。

In [38]:
if step_3:
    dec = {}
    for n in range(n_min,n_max+1):
        dec[n] = GraphDecoder(code[n])

最後に、デコーダオブジェクトを使用して結果を処理できます。ここでは、デフォルトのアルゴリズムである最小重みの完全一致が使用されています。最終結果は、論理エラー確率の計算です。手順3を実行すると、次のスニペットも論理エラー確率を保存します。それ以外の場合は、以前に保存された確率を読み取ります。

In [39]:
if step_3:
    
    logical_prob_match = {}
    for n in range(n_min,n_max+1):
        logical_prob_match[n] = dec[n].get_logical_prob(results[n])
        
    with open('results/logical_prob_match_'+device_name+'.txt', 'w') as file:
        file.write(str(logical_prob_match))
        
else:
    with open('results/logical_prob_match_'+device_name+'.txt', 'r') as file:
        logical_prob_match = eval(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'results/logical_prob_match_ibmq_16_melbourne.txt'

結果の論理エラー確率は、y軸で使用される対数目盛を使用した次のグラフに表示されます。$n$の増加に伴って指数関数的に減衰すると予想されます。これが事実である場合、それはデバイスが量子エラー訂正のこの基本テストと互換性があることの確認です。そうでない場合、それはqubitとゲートが十分に信頼できないことを意味します。

幸い、IBM Qプロトタイプデバイスの結果は通常、予想される指数関数的減衰を示しています。以下の結果では、小さなコードがこのルールの例外を表していることがわかります。コードのサイズの増加に、ノイズが非常に低いまたは高いqubitのグループが含まれる場合など、他の偏差も予想されます。

In [40]:
import matplotlib.pyplot as plt
import numpy as np

x_axis = range(n_min,n_max+1)
P = { log: [logical_prob_match[n][log] for n in x_axis] for log in ['0', '1'] }

ax = plt.gca()
plt.xlabel('Code distance, n')
plt.ylabel('ln(Logical error probability)')
ax.scatter( x_axis, P['0'], label="logical 0")
ax.scatter( x_axis, P['1'], label="logical 1")
ax.set_yscale('log')
ax.set_ylim(ymax=1.5*max(P['0']+P['1']),ymin=0.75*min(P['0']+P['1']))
plt.legend()

plt.show()

NameError: name 'logical_prob_match' is not defined

得られるもう1つの洞察は、結果を使用して、特定のエラープロセスが発生する可能性を判断することです。

これを行うには、シンドロームグラフの各エッジが、回路内の特定のポイントの特定のqubitで発生する特定の形式のエラーを表すという事実を使用します。これは、隣接するノードの両方に対応するシンドローム値を変化させる唯一の単一エラーです。従って、結果を使用してそのようなシンドロームの確率を推定すると、そのようなエラーイベントの確率を推定できます。具体的には、最初にオーダーすると、

$\frac{p}{1-p}≈\frac{C_{11}}{C_{00}}$

ここで、$p$は特定のエッジに対応するエラーの確率、$C_{11}$は結果のカウント数['n']['0']は両方の隣接ノードのシンドローム値が1に対応し、$C_{00}$も同じで0に対応します。

デコーダーオブジェクトには、これらの比率を決定するメソッドweight_syndrome_graphがあり、各エッジに重み$-ln(p/1-p)$を割り当てます。この方法を使用して重みを検査することにより、これらの確率を簡単に取得できます。

In [41]:
if step_3:

    dec[n_max].weight_syndrome_graph(results=results[n_max])

    probs = []
    for edge in dec[n_max].S.edges:
        ratio = np.exp(-dec[n_max].S.get_edge_data(edge[0],edge[1])['distance'])
        probs.append( ratio/(1+ratio) )
        
    with open('results/probs_'+device_name+'.txt', 'w') as file:
        file.write(str(probs))
        
else:
    
    with open('results/probs_'+device_name+'.txt', 'r') as file:
        probs = eval(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'results/probs_ibmq_16_melbourne.txt'

完全なリストを表示するのではなく、平均、標準偏差、最小、最大、四分位数を介してサマリーを取得できます。

In [42]:
import pandas as pd

pd.Series(probs).describe().to_dict()

NameError: name 'probs' is not defined

デバイスのベンチマークでは、全く同じエラー確率のセットは生成されません。ただし、読み出しエラーと制御されたNOTゲートエラーの確率は、優れた比較として役立ちます。具体的には、バックエンドオブジェクトを使用してベンチマークからこれらの値を取得できます。

In [43]:
if step_3:

    gate_probs = []
    for j,qubit in enumerate(line):
        
        gate_probs.append( backend.properties().readout_error(qubit) )
        
        cx1,cx2 = 0,0
        if j>0:
            gate_probs( backend.properties().gate_error('cx',[qubit,line[j-1]]) )
        if j<len(line)-1:
            gate_probs( backend.properties().gate_error('cx',[qubit,line[j+1]]) )
                
    with open('results/gate_probs_'+device_name+'.txt', 'w') as file:
        file.write(str(gate_probs))
        
else:
    
    with open('results/gate_probs_'+device_name+'.txt', 'r') as file:
        gate_probs = eval(file.read())
    
pd.Series(gate_probs).describe().to_dict()

FileNotFoundError: [Errno 2] No such file or directory: 'results/gate_probs_ibmq_16_melbourne.txt'

In [44]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.14.1',
 'qiskit-aer': '0.5.2',
 'qiskit-ignis': '0.3.0',
 'qiskit-ibmq-provider': '0.7.2',
 'qiskit-aqua': '0.7.1',
 'qiskit': '0.19.3'}